In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-nlp") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()
sc = spark.sparkContext

In [2]:
from pyspark.sql.functions import col, countDistinct, desc, year, month, dayofmonth
import datetime
from pyspark.sql.types import DateType

In [3]:
congDF0617 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/June2017.csv/*.part",header=True)
congDF0717 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/July2017.csv/*.part",header=True)
congDF0817 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Aug2017.csv/*.part",header=True)
congDF0917 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Sep2017.csv/*.part",header=True)
congDF1017 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Oct2017.csv/*.part",header=True)
congDF1117 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Nov2017.csv/*.part",header=True)
congDF1217 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Dec2017.csv/*.part",header=True)

In [10]:
congDF1117.show(2)

+---+------------------+-------------+--------+--------------------+--------------------+--------------------+------------------+
|_c0|                id|  screen_name| user_id|                time|                link|                text|            source|
+---+------------------+-------------+--------+--------------------+--------------------+--------------------+------------------+
|  0|930444342602366976| SenBobCorker|73303753|2017-11-14T09:36:...|https://www.twitt...|Today at 10:00 a....|Twitter Web Client|
|  1|930444865590185984|Scotttaylorva|42337890|2017-11-14T09:38:...|https://www.twitt...|I voted no...I be...|Twitter for iPhone|
+---+------------------+-------------+--------+--------------------+--------------------+--------------------+------------------+
only showing top 2 rows



In [5]:
congDF0817.show(2)

+---+------------------+-------------+--------------------+--------------------+--------------------+------------------+---------+
|_c0|                id|  screen_name|                time|                link|                text|            source|  user_id|
+---+------------------+-------------+--------------------+--------------------+--------------------+------------------+---------+
|  0|897565284428648449|lutherstrange|2017-08-15T17:06:...|https://www.twitt...|Watch why @realDo...|     Sprout Social| 15141113|
|  1|897448709507563524|RepRichHudson|2017-08-15T09:23:...|https://www.twitt...|Renee &amp; I joi...|Twitter Web Client|935033864|
+---+------------------+-------------+--------------------+--------------------+--------------------+------------------+---------+
only showing top 2 rows



In [6]:
cong_2017 = sc.union([congDF0617.rdd,congDF0717.rdd,
                  congDF0817.rdd,congDF0917.rdd,
                  congDF1017.rdd,congDF1117.rdd,
                  congDF1217.rdd]).toDF()

In [7]:
cong_2017.show(1)

+---+------------------+-----------+--------------------+--------------------+--------------------+---------+--------+
|_c0|                id|screen_name|                time|                link|                text|   source| user_id|
+---+------------------+-----------+--------------------+--------------------+--------------------+---------+--------+
|  0|879753852312006657| ChrisCoons|2017-06-27T13:30:...|https://www.twitt...|Listen to these s...|TweetDeck|15324851|
+---+------------------+-----------+--------------------+--------------------+--------------------+---------+--------+
only showing top 1 row



In [8]:
cong_2017.groupBy("screen_name").count().orderBy(desc('count')).show(20)

+--------------+-----+
|   screen_name|count|
+--------------+-----+
|          null|37171|
|       auctnr1|10695|
|   RepDonBeyer| 4257|
|    RepJayapal| 3620|
|   RepThompson| 3331|
|  RepCloakroom| 3219|
|   LacyClayMO1| 3058|
|   RosLehtinen| 2386|
|      RoKhanna| 2017|
|    SenateDems| 2011|
|       tedlieu| 2003|
|HispanicCaucus| 1898|
|    JohnCornyn| 1835|
|  keithellison| 1744|
|     WhipHoyer| 1740|
| senorrinhatch| 1727|
|   brianschatz| 1712|
| senrobportman| 1699|
|RepDwightEvans| 1684|
| SenWhitehouse| 1610|
+--------------+-----+
only showing top 20 rows



In [ ]:
cong_2017.groupBy("screen_name").count().orderBy('count').show(10)

In [11]:
congDF0118 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Jan2018.csv/*.part",header=True)
congDF0218 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Feb2018.csv/*.part",header=True)
congDF0318 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Mar2018.csv/*.part",header=True)
congDF0418 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Apr2018.csv/*.part",header=True)
congDF0518 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/May2018.csv/*.part",header=True)
congDF0618 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/June2018.csv/*.part",header=True)
congDF0718 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/July2018.csv/*.part",header=True)
congDF0818 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Aug2018.csv/*.part",header=True)
congDF0918 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Sep2018.csv/*.part",header=True)
congDF1018 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Oct2018.csv/*.part",header=True)
congDF1118 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Nov2018.csv/*.part",header=True)
congDF1218 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Dec2018.csv/*.part",header=True)

In [26]:
congDF1218.show(1)

+-------------------+------------+--------+--------------------+--------------------+--------------------+------------------+
|                 id| screen_name| user_id|                time|                link|                text|            source|
+-------------------+------------+--------+--------------------+--------------------+--------------------+------------------+
|1075891288526983168|SenAlexander|76649729|2018-12-20T18:10:...|https://www.twitt...|The #FarmBill als...|Twitter for iPhone|
+-------------------+------------+--------+--------------------+--------------------+--------------------+------------------+
only showing top 1 row



In [15]:
congDF0118.show(2)

+------------------+-------------+---------+--------------------+--------------------+--------------------+------------------+
|                id|  screen_name|  user_id|                time|                link|                text|            source|
+------------------+-------------+---------+--------------------+--------------------+--------------------+------------------+
|958342797941096449|cathymcmorris| 17976923|2018-01-30T09:15:...|https://www.twitt...|As the #TaxCutsan...|      Media Studio|
|958514743475568640| RepFredUpton|124224165|2018-01-30T20:38:...|https://www.twitt...|#SOTU ready with ...|Twitter Web Client|
+------------------+-------------+---------+--------------------+--------------------+--------------------+------------------+
only showing top 2 rows



In [12]:
cong_2018 = sc.union([congDF0118.rdd,congDF0218.rdd,
                     congDF0318.rdd,congDF0418.rdd,
                     congDF0518.rdd,congDF0618.rdd,
                     congDF0718.rdd,congDF0818.rdd,
                     congDF0918.rdd,congDF1018.rdd,
                     congDF1118.rdd,congDF1218.rdd]).toDF()

In [13]:
cong_2018.groupBy("screen_name").count().orderBy(desc('count')).show(20)

+------------------+-----+
|       screen_name|count|
+------------------+-----+
|              null|66155|
|Twitter Web Client|44468|
|Twitter for iPhone|32719|
|         TweetDeck|22084|
|           auctnr1|19672|
|       LacyClayMO1| 8779|
|       RepDonBeyer| 6004|
|    HispanicCaucus| 5975|
|        RepJayapal| 5087|
|      RepCloakroom| 5029|
|        SenateDems| 4628|
|             CAPAC| 4481|
|           tedlieu| 4217|
|          RoKhanna| 4187|
|        JohnCornyn| 4035|
|     senorrinhatch| 3954|
|        VoteMarsha| 3862|
|       RosLehtinen| 3730|
|   JoaquinCastrotx| 3679|
|       brianschatz| 3642|
+------------------+-----+
only showing top 20 rows



In [27]:
congDF0119 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Jan2019.csv/*.part",header=True)
congDF0219 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Feb2019.csv/*.part",header=True)
congDF0319 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Mar2019.csv/*.part",header=True)
congDF0419 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Apr2019.csv/*.part",header=True)
congDF0519 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/May2019.csv/*.part",header=True)
congDF0619 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/June2019.csv/*.part",header=True)
congDF0719 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/July2019.csv/*.part",header=True)
congDF0819 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Aug2019.csv/*.part",header=True)
congDF0919 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Sep2019.csv/*.part",header=True)
congDF1019 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Oct2019.csv/*.part",header=True)
congDF1119 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Nov2019.csv/*.part",header=True)
congDF1219 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/Dec2019.csv/*.part",header=True)

In [28]:
cong_2019 = sc.union([congDF0119.rdd,congDF0219.rdd,
                     congDF0319.rdd,congDF0419.rdd,
                     congDF0519.rdd,congDF0619.rdd,
                     congDF0719.rdd,congDF0819.rdd,
                     congDF0919.rdd,congDF1019.rdd,
                     congDF1119.rdd,congDF1219.rdd]).toDF()

In [29]:
cong_2019.groupBy("screen_name").count().orderBy(desc('count')).show(20)

+--------------------+------+
|         screen_name| count|
+--------------------+------+
|                null|156526|
|  Twitter for iPhone| 71560|
|     Twitter Web App| 45170|
|  Twitter Web Client| 36748|
|           TweetDeck| 33677|
|             auctnr1| 13196|
|           chiproytx| 10978|
|      HispanicCaucus| 10599|
| Twitter for Android|  6211|
|         LacyClayMO1|  6005|
|         RepDonBeyer|  5922|
|     Castro4Congress|  5828|
|Twitter Media Studio|  5741|
|        RepCloakroom|  5716|
|      HouseJudiciary|  5713|
|      RepAndyBiggsAZ|  5268|
|            HouseGOP|  5195|
|             ewarren|  4830|
|          RepJayapal|  4521|
|       BernieSanders|  4498|
+--------------------+------+
only showing top 20 rows



In [30]:
cong_2019.groupBy("screen_name").count().orderBy('count').show(20)

+--------------------+-----+
|         screen_name|count|
+--------------------+-----+
| Hillary Clinton ...|    1|
| but #BoneSpurs d...|    1|
| &amp; at the end...|    1|
| and to make it e...|    1|
|it would be a dan...|    1|
| with a message t...|    1|
| we will confront...|    1|
| a 7th grade Grot...|    1|
| the White House ...|    1|
| but now a judge ...|    1|
| I led 38 of my c...|    1|
| &amp; we're dema...|    1|
| I can do small t...|    1|
| there was a swee...|    1|
| which @HouseDemo...|    1|
| the unemployment...|    1|
| let us pause to ...|    1|
| y no hay camino ...|    1|
| virtually no ele...|    1|
| Republicans will...|    1|
+--------------------+-----+
only showing top 20 rows



In [9]:
congressDF.show(5)

+-------------------+---------------+-------------------+--------------------+--------------------+--------------------+------------------+
|                _c0|            _c1|                _c2|                 _c3|                 _c4|                 _c5|               _c6|
+-------------------+---------------+-------------------+--------------------+--------------------+--------------------+------------------+
|1207381409809207296|  EliseStefanik|          109579534|2019-12-18T14:25:...|https://www.twitt...|I am excited to d...|Twitter for iPhone|
|1207381362031894537| HouseJudiciary|          246357149|2019-12-18T14:25:...|https://www.twitt...|.@realDonaldTrump...|   Twitter Web App|
|1207381351105732610|CaptClayHiggins|1011053278304591872|2019-12-18T14:25:...|https://www.twitt...|We have a choice,...|   Twitter Web App|
|1207381243408637953|      RepWilson|          234014087|2019-12-18T14:24:...|https://www.twitt...|Today the House v...|   Twitter Web App|
|1207381205999591424

In [5]:
congressDF.agg(countDistinct(col("_c1")).alias("count")).show()

+------+
| count|
+------+
|448622|
+------+



In [14]:
congressDF.groupBy("_c1").count().orderBy(desc('count')).show(20)

+-------------------+------+
|                _c1| count|
+-------------------+------+
|               null|239254|
| Twitter for iPhone|112956|
| Twitter Web Client| 92601|
|          TweetDeck| 63030|
|    Twitter Web App| 45194|
|            auctnr1| 32868|
|     HispanicCaucus| 16574|
|        LacyClayMO1| 14784|
|        RepDonBeyer| 11926|
|          chiproytx| 10978|
|       RepCloakroom| 10745|
|Twitter for Android|  9999|
|         RepJayapal|  9608|
|      Sprout Social|  8706|
|         JohnCornyn|  8500|
|           RoKhanna|  8376|
|           HouseGOP|  7900|
|            tedlieu|  7811|
|     RepAndyBiggsAZ|  7445|
|        brianschatz|  7370|
+-------------------+------+
only showing top 20 rows



In [15]:
congressDF.groupBy("_c6").count().orderBy(desc('count')).show(20)

+--------------------+-------+
|                 _c6|  count|
+--------------------+-------+
|                null|1123464|
|  Twitter for iPhone| 461562|
|  Twitter Web Client| 416847|
|           TweetDeck| 185662|
|     Twitter Web App|  94696|
| Twitter for Android|  30355|
|    Twitter for iPad|  23177|
|Twitter Media Studio|  17758|
|       Sprout Social|  16272|
|        Media Studio|  14382|
|      Hootsuite Inc.|  10594|
|              Buffer|  10109|
|           Hootsuite|   5758|
|        Twitter Lite|   5528|
|           Instagram|   3661|
|            Facebook|   2405|
|Twitter Ads Composer|   1913|
|             Echofon|   1468|
|            Gain App|   1422|
|         Twitter Ads|   1408|
+--------------------+-------+
only showing top 20 rows



In [25]:
congressDF = congressDF.withColumn("date", congressDF['_c3'].cast(DateType()))

In [27]:
congressDF.groupBy("date").count().orderBy(desc('count')).show(100)

+----------+------+
|      date| count|
+----------+------+
|      null|910633|
|2018-01-30|  7389|
|2019-02-05|  6603|
|2019-12-18|  5827|
|2019-11-20|  5022|
|2019-11-13|  4892|
|2019-12-04|  4865|
|2019-12-12|  4678|
|2019-06-12|  4642|
|2019-07-24|  4583|
|2019-12-11|  4355|
|2018-06-20|  4354|
|2019-06-27|  4340|
|2019-04-02|  4325|
|2018-04-17|  4318|
|2019-11-19|  4291|
|2019-06-26|  4285|
|2018-11-06|  4285|
|2019-09-10|  4280|
|2019-02-27|  4264|
|2019-02-26|  4254|
|2019-05-22|  4235|
|2019-10-23|  4234|
|2019-12-10|  4208|
|2019-06-04|  4197|
|2019-06-20|  4173|
|2019-05-16|  4111|
|2019-04-30|  4089|
|2018-06-06|  4060|
|2019-03-27|  4049|
|2018-03-14|  4041|
|2019-06-19|  4040|
|2019-05-08|  4038|
|2019-05-01|  4034|
|2019-10-17|  4025|
|2019-03-26|  4024|
|2019-03-06|  4009|
|2019-02-06|  3973|
|2019-02-07|  3945|
|2019-02-14|  3933|
|2018-05-08|  3933|
|2019-05-15|  3925|
|2019-02-13|  3886|
|2019-03-13|  3883|
|2018-01-19|  3883|
|2018-05-16|  3876|
|2019-07-25|  3847|


In [ ]:
spark.stop()

In [ ]:
sc.stop()